In [1]:
from itertools import repeat
import numpy as np
import numpy.linalg as LA
import sklearn.neighbors
import networkx as nx
from astropy.coordinates import cartesian_to_spherical, spherical_to_cartesian

from craterdetection.matching import database as db
from craterdetection.matching import crater_representation, CoplanarInvariants

In [2]:
df_craters = db.load_craters(path="../data/lunar_crater_database_robbins_2018.csv", diamlims=[4, 30], ellipse_limit=1.1)
df_craters.columns

Index(['CRATER_ID', 'LAT_CIRC_IMG', 'LON_CIRC_IMG', 'LAT_ELLI_IMG',
       'LON_ELLI_IMG', 'DIAM_CIRC_IMG', 'DIAM_CIRC_SD_IMG',
       'DIAM_ELLI_MAJOR_IMG', 'DIAM_ELLI_MINOR_IMG', 'DIAM_ELLI_ECCEN_IMG',
       'DIAM_ELLI_ELLIP_IMG', 'DIAM_ELLI_ANGLE_IMG', 'LAT_ELLI_SD_IMG',
       'LON_ELLI_SD_IMG', 'DIAM_ELLI_MAJOR_SD_IMG', 'DIAM_ELLI_MINOR_SD_IMG',
       'DIAM_ELLI_ANGLE_SD_IMG', 'DIAM_ELLI_ECCEN_SD_IMG',
       'DIAM_ELLI_ELLIP_SD_IMG', 'ARC_IMG', 'PTS_RIM_IMG'],
      dtype='object')

In [3]:
Rplanet = 1737.1
lat, long = df_craters[['LAT_ELLI_IMG', 'LON_ELLI_IMG']].to_numpy().T
lat, long = map(np.radians, (lat, long)) # ALWAYS CONVERT TO RADIANS
psi = np.radians(df_craters['DIAM_ELLI_ANGLE_IMG'].to_numpy())
major, minor = df_craters[['DIAM_ELLI_MAJOR_IMG', 'DIAM_ELLI_MINOR_IMG']].to_numpy().T
x, y, z = db.latlong2cartesian(lat, long)

connectivity_matrix = sklearn.neighbors.radius_neighbors_graph(np.array([x, y, z]).T, 200,
                                                               mode='connectivity',
                                                               metric='euclidean', n_jobs=-1)

In [4]:
G = nx.from_scipy_sparse_matrix(connectivity_matrix)

In [5]:
triangles = np.array([c for c in nx.cycle_basis(G) if len(c)==3])

In [6]:
# Calculate centroid for each crater triad
def triad_splice(arr, triangles_):
    return np.array((arr[triangles_[:, 0]], arr[triangles_[:, 1]], arr[triangles_[:, 2]]))

def swap_np_columns(arr_):
    arr = arr_.copy()
    arr[:, 0], arr[:, 1] = arr[:, 1], arr[:, 0].copy()
    return arr

In [7]:
avg_triad_x, avg_triad_y, avg_triad_z = map(lambda c: np.sum(triad_splice(c, triangles), axis=0)/3., (x, y, z))
_, avg_triad_lat, avg_triad_long = cartesian_to_spherical(avg_triad_x, avg_triad_y, avg_triad_z)
avg_triad_lat, avg_triad_long = map(np.array, (avg_triad_lat, avg_triad_long))

dlat = np.array(triad_splice(lat, triangles)) - np.tile(avg_triad_lat, (3, 1))
dlong = np.array(triad_splice(long, triangles)) - np.tile(avg_triad_long, (3, 1))

x_triads = np.array([2 * Rplanet * np.arcsin(np.cos(np.radians(avg_triad_lat)) * np.sin(dlong_i / 2)) for dlong_i in dlong])
y_triads = np.array([Rplanet * dlat_i for dlat_i in dlat])

In [8]:
# https://math.stackexchange.com/questions/1324179/how-to-tell-if-3-connected-points-are-connected-clockwise-or-counter-clockwise
clockwise = LA.det(np.moveaxis(np.array([[x_triads[0], y_triads[0], np.ones_like(x_triads[0])],
                                        [x_triads[1], y_triads[1], np.ones_like(x_triads[0])],
                                        [x_triads[2], y_triads[2], np.ones_like(x_triads[0])]]), -1, 0)) < 0

triangles_cw = triangles.copy()
triangles_cw[~clockwise] = swap_np_columns(triangles[~clockwise])

avg_triad_x, avg_triad_y, avg_triad_z = map(lambda c: np.sum(triad_splice(c, triangles_cw), axis=0)/3., (x, y, z))
_, avg_triad_lat, avg_triad_long = cartesian_to_spherical(avg_triad_x, avg_triad_y, avg_triad_z)
avg_triad_lat, avg_triad_long = map(np.array, (avg_triad_lat, avg_triad_long))

dlat = np.array(triad_splice(lat, triangles_cw)) - np.tile(avg_triad_lat, (3, 1))
dlong = np.array(triad_splice(long, triangles_cw)) - np.tile(avg_triad_long, (3, 1))

x_triads = np.array([2 * Rplanet * np.arcsin(np.cos(np.radians(avg_triad_lat)) * np.sin(dlong_i / 2)) for dlong_i in dlong])
y_triads = np.array([Rplanet * dlat_i for dlat_i in dlat])
a_triads, b_triads, psi_triads = map(triad_splice, (major, minor, psi), repeat(triangles_cw))

In [9]:
# True if all crater triads are clockwise
np.logical_and.reduce(LA.det(np.moveaxis(np.array([[x_triads[0], y_triads[0], np.ones_like(x_triads[0])],
                                        [x_triads[1], y_triads[1], np.ones_like(x_triads[0])],
                                        [x_triads[2], y_triads[2], np.ones_like(x_triads[0])]]), -1, 0)) < 0)

True

In [10]:
crater_triads_matrices = []
for args in zip(x_triads, y_triads, a_triads, b_triads, psi_triads):
    crater_triads_matrices.append(crater_representation(*args))
A_i, A_j, A_k = np.array(crater_triads_matrices)

In [12]:
invariants = CoplanarInvariants(triangles_cw, A_i, A_j, A_k, normalize_det=True)

In [14]:
invariants.get_pattern().shape

(4404219, 7)